In [1]:
# XML, TEI, CMI/F and data handling
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/) for XML
from bs4 import Comment # BS4-addon for å håndtere kommentarer <!-- X -->
import re # Regex
import pandas as pd
import collections # Facilitate dynamic dict

# Time and date
import datetime # Dates
from datetime import date
import time # Time

# File and folder handling
import glob # The yeast of thought and mind
import os # Filsystem; mapper, lagring, åpning, etc...
import shutil # Se os+
import json # JSON!

from string import punctuation

#Check if *keys (nested) exists in dict
def keys_exists(element, *keys):
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True

today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = "MXMLM Preprocessor"
print("Version",currVer)
cmifUid = "a403c593-09df-4538-8acf" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.
# cmifUid is also used as "source" for the time being in each object. Read more about this in CMIF docs.

# *MunchXMLmuncher* **X2**
# MXMLMx2 reads and preprocesses files. First off, create subdirectories and locate our files:

Version MXMLM Preprocessor


In [2]:
hasChrono,hasXMLs = False,False

In [3]:
# Chrono
# Next, let's get the chrono if it exists, and then read the data out of it to a dict.

# Chronology

In [4]:
listofbaddies = []
lookupChrono = sorted(glob.glob("*Kronologi_Munchs_brev*.xlsx"), key=os.path.getmtime)
x = len(lookupChrono)-1
if x > -1:
    print("Newest chronology file:",lookupChrono[x])
    chronologyFile = lookupChrono[x]
    hasChrono = True
    #shutil.copy2(chronologyFile, inputfolder+"/kronologi.xlsx")
    #chronologyFile = inputfolder+"/kronologi.xlsx"
    
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    
    try:
        if CHRONODICT:
            print("CHRONODICT found with content")
        else:
            print("CHRONODICT found without content")
    except:
        CHRONODICT = collections.defaultdict(dict)
        print("CHRONODICT created")
    for idx,row in chronology.iterrows():
        formattingError = False
        document = chronology.iloc[idx]['Objektnr.']
        rawdate = chronology.iloc[idx]['Dato']
        if rawdate != "N/A" and document != "N/A": # If date and documents are not N/A
            chkStr = document[0:2]
            if chkStr == "PN":
                #print(document)
                document = document.replace(" ","0")
                if len(document) == 5:
                    filenameEdited=document[:2] + "0" + document[len(document)-3:]
                else:
                    filenameEdited = document[:2] +"000"+ document[len(document)-4:]
                    filenameEdited = filenameEdited[:2] + filenameEdited[len(filenameEdited)-4:]
                filenamePlain = filenameEdited[0:6]
            elif chkStr == "MM":
                if "," in document or "+" in document: # If the document title has a comment in it with either of these...
                    #print("> Breaking",document)
                    if "," in document:
                        breakup = document.split(",")
                    elif "+" in document:
                        breakup  = document.split("+")
                    document = breakup[0] # Break the title and try to use the first part onwards.
                
                #No-MM_K0499
                #filenamePlain = document[0:11]
                if len(document) == 8:
                    #print(document)
                    filenamePlain = document[:2]+"_"+document[3]+"0"+document[5:]
                elif len(document) == 10:
                    filenamePlain = document[:2]+"_"+document[3]+document[6:]
                else:
                    filenamePlain = document[:2]+"_"+document[3]+document[5:]
                filenamePlain = "No-"+filenamePlain
                #print(document,len(document),"/",filenamePlain,len(filenamePlain))
            elif chkStr == "M ": # Implemented to catch "M N 2714"...
                filenamePlain = document[:1]+"_"+document[2]+document[4:]
                filenamePlain = "No-M"+filenamePlain
                #filenamePlain = filenamePlain.replace(" ","")
                #print("Caught",document,">",filenamePlain)
            elif chkStr == "N ": # Implemented to catch "N 3491"...
                  # This is ASSUMED to be "No-MM_N3491" and not PN3491
                filenamePlain = "No-MM_N"+document[2:]
                #print("Caught",document,">",filenamePlain)
            elif chkStr == " M": # Implemented to catch " MM N 723"...
                filenamePlain = "No-"+document[1:3]+"_"+document[4]+"0"+document[6:]
                #print("Caught",document,">",filenamePlain)
            else:
                print("!!\tError in",document,"length:",len(document))
                #filenamePlain = "Formatting error"+document
                formattingError = True # Formatting error due to invalid filename.
            #print(filenamePlain)
            if len(filenamePlain) != len("No-MM_N0000"): # If string doesn't match with No-MM_N0000
                if len(filenamePlain) != len("PN0000"): # If string doesn't match with PN0000
                    #filenamePlain = "Formatting error"+filenamePlain # it is invalid.
                    formattingError = True # Formatting error due to invalid filename.
                    listofbaddies.append(filenamePlain+" ("+document+")")
            document = filenamePlain
            if isinstance(rawdate,datetime.date): # If it's just a datetime object
                #dateobject = rawdate.strftime("%Y-%m-%d")
                newdate = rawdate.strftime("%Y-%m-%d")
            else:
                #print(document,rawdate)
                dateobject = str(rawdate) # Make sure it's string
                
                string4print = document+" "+dateobject
                
                dateobject = dateobject.replace("–","-") # Replace long dash – with a normal dash -
                if ".-" in dateobject: # Like: 04.-05.1922.
                    dateobject = re.sub(r'^.*?\.-', '', dateobject, flags=re.DOTALL) # Remove EVERYTHING to the left of it.
# If you want date ranges, use complete dates separated by "-".
# Using ".-" to denote several things is not good.
                if "?" in dateobject:
                    dateobject = dateobject.replace("?","").strip(punctuation) # Remove n ?s and then also remove excess .

                    dateobject = dateobject.replace("..",".")
                if "-" in dateobject:
                    splitToFrom = dateobject.split("-")
                    fromDate = splitToFrom[0]
                    toDate = splitToFrom[1]
                    newFromDate = fromDate.split(".")
                    if isinstance(newFromDate,list):
                        newdate = newFromDate[len(newFromDate)-1]
                        for x in reversed(newFromDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdate+="-"+x
                                else:
                                    break  
                    else:
                        newdate = fromDate
                    newdate+="%"
                    newToDate = toDate.split(".")
                    if isinstance(newToDate,list):
                        newdateA = newToDate[len(newToDate)-1]
                        for x in reversed(newToDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdateA+="-"+x
                                else:
                                    break  
                    else:
                        newdateA = toDate
                    newdate+=newdateA
                    #print("yolo",document)
                else:
                    datelements = dateobject.split(".")
                    newdate = datelements[len(datelements)-1]
                    for x in reversed(datelements):
                        if len(x) == 4:
                            pass
                        else:
                            if len(x) == 2:
                                newdate+="-"+x
                            else:
                                break      
                #print(document,rawdate,"("+newdate+")")
            if formattingError is True:
                print(document,"is not a valid document ID and was excluded.")
            else:
                CHRONODICT[document]["date"] = newdate # Set the dict item's date to newdate

Newest chronology file: Kronologi_Munchs_brev_20220923.xlsx
CHRONODICT created
No-MM_K? is not a valid document ID and was excluded.
No-MM_K99 is not a valid document ID and was excluded.
No-MM_K27 is not a valid document ID and was excluded.
No-MM_N2 is not a valid document ID and was excluded.


# Placenames

In [5]:
## Places
# With the dates safely stored away, let's go for some place names.
if os.path.isfile("ID_sted-verdier.xlsx"):
    print("Detected ID_sted-verdier.xlsx")
    if os.path.exists("xml-filer"):
        print("XML>CMIF placename augmentation enabled")
        listXMLfiles = glob.glob("xml-filer/**/*.xml",recursive=True)
        hasXMLs = True
    else:
        print("No XML files provided. XML>CMIF placename augmentation disabled")
else:
    print("No ID_sted-verdier file provided. Skipping placename augmentation.")

Detected ID_sted-verdier.xlsx
XML>CMIF placename augmentation enabled


In [6]:
#n = 0
if hasXMLs == True: # If the XML files should be used for updating
    # Check to see if CHRONODICT is alive or not. If it is, use it as destination. If it isn't, create it.
    try:
        if CHRONODICT:
            print("CHRONODICT found with content")
        else:
            print("CHRONODICT found without content")
    except:
        CHRONODICT = collections.defaultdict(dict)
        print("CHRONODICT created")

    addrsFoundInXMLs = [] # Make a simple list to hold the short names of every file we've found addresses for
    xmlswithnoaddress = [] # Simple list for XMLs that have no address that can be printed later.
    plainnames = [] # Simple list for plain names of XMLs that have been checked.
    i=0
    for item in listXMLfiles:
        addrKey = "NONE" # Just in case
        find_address = [] # Ensure that this list clears on start of each item
        findFileName = item.split("\\") # Make filepath a list
        findFileName = findFileName[len(findFileName)-1] # Get the path destination file
        
        chkStr = findFileName[0:2] # Check the incoming ID - it's either PN+4 positions, or No+9 positions long.
        if chkStr == "PN": # 6 positions
            filenamePlain = findFileName[0:6] # Must be bounded to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        elif chkStr == "No": # 11 positions
            filenamePlain = findFileName[0:11] # Must be bounded to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        else: # This doesn't occur unless you've got files that don't belong here
            print("PROBLEM IN NAME PROCESSING",findFileName,chkStr) 
        if filenamePlain not in plainnames: # For every unique name, add to plainnames
            plainnames.append(str(filenamePlain)) # Just in case we need them later
        print ("\r","Progress:",round(i/len(listXMLfiles)*100),"%", end='\t')
        #print ("\r",n*0.1, end='')
        #n+=1
        if filenamePlain in addrsFoundInXMLs: # if we found the address for this xml
            pass # Skip if we've already found an address for this XML filename
        else:
            with open(item, "r", encoding="utf-8") as file: # Open a file
                letterfile = file.readlines() # Les innholdet som linjer
                letterfile = "".join(letterfile) # Linjene blir kombinert i en variabel
            soup = BeautifulSoup(letterfile, features="xml") # It is now soup
            #find_address = soup.findAll("address") # Look for addrline element.
            #augh,addrKey = "","" # Reset
            foundSender,foundRecipient = False,False # Reset

            ## Code below enables retrieval of an address enclosed in a dateline element.
            ## This is understood to be the sender's address.
            if foundSender == False: # If a recipient has not been found
                find_address = soup.find("dateline") # Look for a dateline element
                if find_address: # If there is a dateline element:
                    #print("Dateline in",findFileName)
                    find_address = find_address.findChild("placeName", recursive=True) # Get the placename
                    #print("DATELINE",find_address)
                    try: # There are documents with datelines but no locations in them confirmed.
                        addrKey = find_address.get('key') # Get the internal ID of the placename
                        try:
                            addrKey = addrKey.replace("pl","") # Remove "pl" prefix
                            addrsFoundInXMLs.append(filenamePlain) # Add the filename to the list of XMLs already found
                            #placenamedict[filenamePlain]["location"] = addrKey
                            CHRONODICT[filenamePlain]["location"] = addrKey
                            foundSender = True
                        except:
                            print("Problem with appending")

                    except:
                        if filenamePlain not in xmlswithnoaddress:
                            xmlswithnoaddress.append(filenamePlain)
                        #print("Dateline without address in",findFileName)
                else:
                    if filenamePlain not in xmlswithnoaddress:
                        xmlswithnoaddress.append(filenamePlain)
        i+=1
print("\t")

CHRONODICT found with content
 Progress: 100 %		 %	%	11 %	%	%	%	%	%		%				 %	 %	%	27 %	%		%	%		31 %	 %	%	%	%		%			%	 %	 %	%		60 %			67 %	67 %	%	69 %						%									


In [7]:
if hasXMLs == True:
    placenamedf = pd.read_excel("ID_sted-verdier.xlsx").dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    placenamedf = placenamedf.fillna("N/A")
    #for item in placenamedict:
    for item in CHRONODICT:
        if keys_exists(CHRONODICT,item,'location'):
            #print("\tSent from:",CHRONODICT[item]['location'])
            #print(item)
            try:
                placenameSearch = placenamedf[placenamedf['ID'].astype(str) == str(CHRONODICT[item]['location'])]
                stedsnavn = placenameSearch["sted"].values[0]
                regionnavn = placenameSearch["region, nasjonal"].values[0]
                landnavn = placenameSearch["land"].values[0]
                kontinent = placenameSearch["region, internasjonal"].values[0]
                try:
                    if stedsnavn != "N/A":
                        #print("\tSted:",stedsnavn)
                        CHRONODICT[item]['location'] = stedsnavn
                    else:
                        if regionnavn != "N/A":
                            #print("\tRegion:",regionnavn)
                            CHRONODICT[item]['location'] = regionnavn
                        else:
                            if landnavn != "N/A":
                                #print("\tLand:",landnavn)
                                CHRONODICT[item]['location'] = landnavn
                            else:
                                if kontinent != "N/A":
                                    #print("\tKontinent:",kontinent)
                                    CHRONODICT[item]['location'] = kontinent
                                else:
                                    print("\tFant ikke stedsnavn.")
                except:
                    print("\t\tCouldn't match, I guess?")
            except:
                print("\t\tNo match for",str(CHRONODICT[item]['location']))
        #print("\n")

In [8]:
df = pd.DataFrame.from_dict(CHRONODICT)
chronoDF = df.T.fillna("N/A").reset_index(drop=False)
# Check for presence of date column (otherwise, would crash if no chronology used)
if 'date' not in chronoDF.columns:
    chronoDF['date'] = np.nan
    print("Filled date column with NaN")
# Check for presence of location column (otherwise, would crash if no letter XMLs used)
if 'location' not in chronoDF.columns:
    chronoDF['location'] = np.nan
    print("Filled location column with NaN")
chronoDF = chronoDF.rename(columns={"index": "document"}).sort_values(by=['document']).reset_index(drop=True)
places = 0
dates = 0
items = 0
for idx,row in chronoDF.iterrows():
    if row['date'] == "N/A" and row['location'] == "N/A":
        print(idx)
    else:
        #print(idx)
        items += 1
        if row['date'] != "N/A":
            #print(row['date'])
            dates += 1
        if row['location'] != "N/A":
            #print(row['location'])
            places += 1
        #print("-")
print(places,"places and",dates,"dates added over a total of",items,"items.")
chronoDF.to_csv("preprocessed.csv", sep=',', encoding='utf-8',index=False)
print("Preprocessing complete. Saved to preprocessed.csv. Printing report as Preprocessor Report.txt.")

if i > 0:
    pass
else:
    i=0

goodstring = "MXMLM Preprocessor run on date "+str(today)+"."
try:
    if plainnames:
        goodstring += "\nChecked "+str(i)+" files, of which "+str(len(plainnames))+" were identified as letters, of which "+str(len(addrsFoundInXMLs))+" had addresses."
    else:
        goodstring += "\nCAUTION Checked "+str(i)+" (zero?) files, of which "+str(len(plainnames))+" were identified as letters, of which "+str(len(addrsFoundInXMLs))+" had addresses."
except:
    goodstring+= "\nDidn't use placename augmentation?"
goodstring+="\n"+str(places)+" places and "+str(dates)+" dates added over a total of "+str(items)+" items."
if len(listofbaddies) > 0:
    errorstring = "Bad document IDs from the Chronology that could not be resolved:\n"
    for x in listofbaddies:
        errorstring+="\""+x+"\" "
    errorstring = errorstring.rstrip()
else:
    errorstring = "Didn't identify any bad document IDs that couldn't be resolved."
outputstring = goodstring+"\n"+errorstring

with open("Preprocessor Report.txt", "w", encoding="utf-8") as output_file:
    output_file.write(outputstring)

498 places and 2654 dates added over a total of 2814 items.
Preprocessing complete. Saved to preprocessed.csv. Printing report as Preprocessor Report.txt.


# Correspondence

In [9]:
## Main file: Correspondence
# Decrypting XML files is slow, and doing it while constructing an XML is unneccessarily complex.
# Let's just process the XML files here.

# Because Correspondence and Register_Tei are slightly different, I'm writing specific code for each despite
# the fact that it's somewhat WET programming.

if os.path.isfile("correspondence.xml"):
    CorrespDict = collections.defaultdict(dict)
    print("CorrespDict initiated")
    print("Melting correspondence.xml")
    with open("correspondence.xml", "r", encoding="utf-8") as file: # Open a file
        tei = file.readlines() # Les innholdet som linjer
        tei = "".join(tei) # Linjene blir kombinert i en variabel
    soup = BeautifulSoup(tei, from_encoding="UTF-8", features="xml") # It is now soup
    # Don't worry about the error screaming about Unicode markup being provided twice
    print("Correspondence is now soup.")
    comments = 0
    commentDocs = 0
    for comment in soup.findAll(string=lambda text: isinstance(text, Comment)):
        if "xml:id=\"" in comment:
            commentDocs+=1
        comment.extract()
        comments+=1
    if comments > 0:
        print("Destroyed",comments,"<!--comments-->, of which",commentDocs,"contained an @XML:ID.")
    # ... and checking it twice.
    comments = soup.findAll(string=lambda text: isinstance(text, Comment))
    if comments:
        print("There are still",len(comments),"comments present.")
    else:
        print("All comments destroyed.")
        
    print("\nInitializing documentID scan.")
    documentIDs = []
    for document in soup.findAll("div", {"xml:id":True}):

        # Look for the document type assignment.
        documentType = document.find("list", {"type" : "objectType"}).findChild(True, recursive=True)#.attrs['n']
        # Checks if the words "letter" or "brev" appear in the type
        if "brev" in documentType or "letter" in documentType: 
            # Get the document ID from the <div> element.
            documentID = list(document.attrs.values())[0]
            documentIDs.append(documentID)
    print("Acquired",len(documentIDs),"documents classed as letters.\n")
        
    # Slight repetition - breaking DRY, I know - but by referencing documentIDs we're 100% only treating letters,
    # not wasting time on irrelevant documents
    i=1
    for eachID in documentIDs:
        
        i+=1
        #MM_K3421
        docAuthors = [] # List of authors to be included in the dict.
        docAuthorRefs = [] # List of authors' reference URLs.
        docRecipients = [] # List of recipients to be included in the dict. 

        #print(eachID)
        # Munch is the recipient of everything in correspondence.xml.
        recipient = "Edvard Munch"

        # Target the document as var "document"
        document = soup.find("div", {"xml:id":eachID})

        # Target the author(s) as authorNameList
        authorNameList = document.find("item", {"n":"sender"}).findChildren(True, recursive=True)
        X=0
        for name in authorNameList:
            try:
                authorName = authorNameList[X].contents[0]
                try:
                    targetRef = authorName['target']
                except:
                    targetRef = "N/A"
            except:
                authorName = "N/A"
                targetRef = "N/A"
            
            # Data cleaning
            authorName = authorName.replace(","," ")
            #authorName = authorName.replace(";"," ")
            #authorName = authorName.replace("["," ")
            #authorName = authorName.replace("]"," ")
            #authorName = authorName.replace("?"," ")
            authorName = re.sub(' +', ' ',authorName)
            authorName = authorName.strip()
            
            
            X+=1
            docAuthors.append(authorName)
            docAuthorRefs.append(targetRef)

        statusMessage = str(eachID)+" "+str(len(authorNameList))+" author"
        if len(docAuthors) > 1:
            statusMessage += "s"
        statusMessage+=" ("
        z = 1
        for author in docAuthors:
            if author != "N/A":
                statusMessage+=str(author)
            else:
                statusMessage+="N/A (error?)"
            if z < len(docAuthors):
                statusMessage+=","
            z+=1
        statusMessage+=") addressed to: "+str(recipient)


        isDocumentUndated = document.find("item", {"n":"undated"})
        if isDocumentUndated:
            # Document is straight up undated.
            date = "s.d."
            datetype = "N/A"

        else:
            #statusMessage+="\n>Dated: "
            isDocumentFromTo = document.find("date", {"from":True}) # Does the date element have a from assignment? 
            # ! Using "from" because PN1350 does not have a fromTo attr despite using fromTo. Uses "from", though. Works fine.
            if isDocumentFromTo: # If it does, and thus has a range (JK, No-MM_T1296 has FROM attr but not a TO attr.)
                doesDocumentHaveToDate = document.find("date", {"to":True})
                #statusMessage+="range, "
                if doesDocumentHaveToDate:
                    # Both from and to attributes are present.
                    fromDate = isDocumentFromTo['from'] # Extract 'from' date. 
                    toDate = isDocumentFromTo['to'] # Extract 'to' date.
                    datetype = "fromTo"
                    date = str(fromDate)+"%"+str(toDate)
                    #statusMessage+=datetype+" "+str(fromDate)+"-"+str(toDate)
                else:
                    # If the 'from' attribute is present without the 'to', it's interpreted as "not before this date".
                    date = isDocumentFromTo['from']
                    fromDate = isDocumentFromTo['from']
                    datetype = "notBefore"

            else: # If it doesn't:
                #statusMessage+="instance, "
                yearSent = document.find("date", {"type":"year","when":True}) # Check for year element
                monthSent = document.find("date", {"type":"month","when":True}) # Check for month element
                daySent = document.find("date", {"type":"day","when":True}) # Check for day element
                if yearSent:
                    datetype = "instance"
                    date = yearSent.attrs["when"]
                    if monthSent: # Only look for a month if there's a year. That 1 letter with just month/day, tho...
                        M = re.sub('[-]', '', monthSent.attrs["when"]) # Strip the random '-' characters in here.
                        date+="-"+str(M) # Join month to year by YYYY-MM.
                        if daySent: # Only applies if there is a month AND a day. No point having a day if you don't have a month.
                            M = re.sub('[-]', '', daySent.attrs["when"]) # Strip the random '-' characters in here, too.
                            date+="-"+str(M) # Join day to year-month by YYYY-MM-DD.

                else: 
                # If it doesn't have a year, make one last check
                    doesDocumentHaveToDate = document.find("date", {"to":True}) # if the date just has a to date...

                    if doesDocumentHaveToDate:
                    # If the 'to' attribute is present without the 'from', it's interpreted as "not after this date".
                        datetype = "notAfter"
                        date = doesDocumentHaveToDate['to']

                    else:
                    # All else has failed. This data is expunged.
                        datetype = "N/A"
                        date = "s.d."


        #statusMessage+="\n"
        if datetype == "N/A":
            statusMessage+="\n(undated)"
        else:
            statusMessage+="\nDate type: "+str(datetype)+" ("+str(date)+")"
        print(statusMessage)
        CorrespDict[eachID]['authors'] = docAuthors
        CorrespDict[eachID]['date'] = date
        CorrespDict[eachID]['datetype'] = datetype
        CorrespDict[eachID]['recipients'] = recipient

        json_object = json.dumps(CorrespDict, indent=4)

        with open("correspondence.json", "w") as outfile:
            outfile.write(json_object)

else:
    print("No correspondence.xml file provided. MXML will not munch letters to Munch.")

CorrespDict initiated
Melting correspondence.xml


C:\Users\lokes\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Correspondence is now soup.
Destroyed 15 <!--comments-->, of which 7 contained an @XML:ID.
All comments destroyed.

Initializing documentID scan.
Acquired 5816 documents classed as letters.

No-MM_K0001 1 author (Aagot Andersen) addressed to: Edvard Munch
Date type: notBefore (1924)
No-MM_K0002 1 author (A. E. Andersen) addressed to: Edvard Munch
Date type: instance (1918-02-27)
No-MM_K0003 1 author (R. M. Arbo) addressed to: Edvard Munch
Date type: instance (1889-04-23)
No-MM_K0004 1 author (M. B. Arnet-Johnson) addressed to: Edvard Munch
Date type: instance (1933-12-16)
No-MM_K0005 1 author (Ola Abrahamsson) addressed to: Edvard Munch
Date type: notAfter (1925)
No-MM_K0006 1 author (Ola Abrahamsson) addressed to: Edvard Munch
Date type: instance (1914-11-19)
No-MM_K0007 1 author (Ola Abrahamsson) addressed to: Edvard Munch
Date type: instance (1913-08-22)
No-MM_K0008 1 author (Ola Abrahamsson) addressed to: Edvard Munch
Date type: instance (1928-11-04)
No-MM_K0009 1 author (Ola Abrah

No-MM_K0091 1 author (Lucien Dedichen) addressed to: Edvard Munch
(undated)
No-MM_K0092 1 author (Jenny Dedichen) addressed to: Edvard Munch
(undated)
No-MM_K0093 1 author (Jenny Dedichen) addressed to: Edvard Munch
(undated)
No-MM_K0094 1 author (Dyre Diriks) addressed to: Edvard Munch
Date type: instance (1929-04-20)
No-MM_K0096 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1890-11-11)
No-MM_K0097 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1890-11-26)
No-MM_K0098 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1890-12-06)
No-MM_K0099 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1903-06-12)
No-MM_K0100 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1903-07-25)
No-MM_K0101 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1904-05-07)
No-MM_K0102 1 author (Edvard Diriks) addressed to: Edvard Munch
Date type: instance (1912-04-26)


No-MM_K0175 1 author (Eline Finne) addressed to: Edvard Munch
Date type: instance (1939-12-12)
No-MM_K0176 1 author (Lily Gallostra) addressed to: Edvard Munch
Date type: fromTo (1933%1933)
No-MM_K0177 2 authors (Ola Geelmuyden,Christiania Kunstforening) addressed to: Edvard Munch
Date type: instance (1921-04-28)
No-MM_K0178 1 author (M. Gjermundsen) addressed to: Edvard Munch
Date type: instance (1912-08-28)
No-MM_K0179 1 author (Inger Gløersen) addressed to: Edvard Munch
Date type: instance (1924-09-08)
No-MM_K0180 1 author (Ove Wahl Graff) addressed to: Edvard Munch
Date type: instance (1925-04-03)
No-MM_K0181 1 author (Fr. Grøn) addressed to: Edvard Munch
Date type: instance (1915-10-19)
No-MM_K0182 1 author (Gabriel Gustafson) addressed to: Edvard Munch
(undated)
No-MM_K0183 1 author (Gyda Gaarder) addressed to: Edvard Munch
Date type: instance (1926-08-26)
No-MM_K0184 1 author (Pola Gauguin) addressed to: Edvard Munch
Date type: instance (1933-09-22)
No-MM_K0185 1 author (Pola Ga

...

No-MM_K5515 2 authors (H. Romstad,Narvesens Kioskkompani Bokhandel og Avissalg ved Jernbanerne) addressed to: Edvard Munch
Date type: instance (1913-05-20)
No-MM_K5516 1 author (Norges Post) addressed to: Edvard Munch
Date type: instance (1918-01-12)
No-MM_K5517 2 authors (Høydahl Ohme,Høydahl Ohme's Annonce-Expedition) addressed to: Edvard Munch
Date type: instance (1918-04-22)
No-MM_K5518 1 author (Scheteligs Bokhandel) addressed to: Edvard Munch
Date type: instance (1939-02-28)
No-MM_K5519 1 author (Scheteligs Bokhandel) addressed to: Edvard Munch
Date type: instance (1938-09-27)
No-MM_K5520 1 author (N. Torkilsen) addressed to: Edvard Munch
Date type: instance (1918-08-01)
No-MM_K5521 2 authors (S.H. Lundh,S. H. Lundh & Co. A/S Norges ældste og største Forretning i Landbruksmaskiner og Redskaper) addressed to: Edvard Munch
(undated)
No-MM_K5522 2 authors (Svern Pedersen,Grøndahls Flygel- og Pianolager A/S) addressed to: Edvard Munch
Date type: instance (1926-03-10)
No-MM_K5523 2 au

No-MM_K5585 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-05-27)
No-MM_K5586 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-06-02)
No-MM_K5587 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-06-21)
No-MM_K5588 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-06-17)
No-MM_K5589 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-07-15)
No-MM_K5590 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-07-04)
No-MM_K5591 1 author (konserves- & frugt Jensen & Co Fetevarer. Delikatesser) addressed to: Edvard Munch
Date type: instance (1919-07-05)
No-MM_K5592 1 author (konserves- &

No-MM_K5647 2 authors (Hans Andersen,Hans Andersen) addressed to: Edvard Munch
Date type: instance (1910-10-23)
No-MM_K5649 1 author (Farris - Fritzoe) addressed to: Edvard Munch
Date type: instance (1937-11-17)
No-MM_K5650 2 authors (Olsen,ukjent) addressed to: Edvard Munch
(undated)
No-MM_K5651 2 authors (H.? Synersen,H. Synersen) addressed to: Edvard Munch
Date type: instance (1912-09-16)
No-MM_K5652 1 author (Hagbarth Steffens) addressed to: Edvard Munch
Date type: instance (1927-06-15)
No-MM_K5653 1 author (Moss Privatbank Elektricitetsverk Driftsbestyreren) addressed to: Edvard Munch
Date type: instance (1915-02-22)
No-MM_K5654 2 authors (R. Thorsen,Telegraf- og telefonkassereren) addressed to: Edvard Munch
Date type: instance (1939-01-26)
No-MM_K5655 1 author (Oslo Telefon Anlegg) addressed to: Edvard Munch
Date type: instance (1938-11)
No-MM_K5656 2 authors (P. Pedersen,Det Norske Telegrafvesen Rikstelefonstasjonen) addressed to: Edvard Munch
Date type: instance (1938-07-20)
No

No-MM_K5725 1 author (Nanna Thaning) addressed to: Edvard Munch
Date type: instance (1916-12-30)
No-MM_K5726 1 author (Curt Glaser) addressed to: Edvard Munch
Date type: instance (1915-11-19)
No-MM_K5727 1 author (Curt Glaser) addressed to: Edvard Munch
Date type: instance (1939-12-28)
No-MM_K5728 1 author (Curt Glaser) addressed to: Edvard Munch
(undated)
No-MM_K5729 2 authors (Wilhelm Felsing,Hof-Kunstkupferdruckerei O. Felsing) addressed to: Edvard Munch
Date type: instance (1913-01-25)
No-MM_K5730 2 authors (Wilhelm Felsing,Hof-Kunstkupferdruckerei O. Felsing) addressed to: Edvard Munch
Date type: instance (1905-02-22)
No-MM_K5731 1 author (Klose & Seidel Bureau für zeitungs-ausschnitte) addressed to: Edvard Munch
Date type: instance (1915-11-29)
No-MM_K5732 1 author (Walther Rathenau) addressed to: Edvard Munch
Date type: instance (1931-06-24)
No-MM_K5733 2 authors (Gustav Gerstenberger,Gustav Gernstenberger) addressed to: Edvard Munch
Date type: instance (1928-11-28)
No-MM_K5734 

No-MM_K5810 1 author (Anna Henrichsen) addressed to: Edvard Munch
(undated)
No-MM_K5811 1 author (Uidentifisert person) addressed to: Edvard Munch
(undated)
No-MM_K5812 1 author (Uidentifisert person) addressed to: Edvard Munch
Date type: instance (1940-12-12)
No-MM_K5813 1 author (Kunstnerforbundet) addressed to: Edvard Munch
(undated)
No-MM_K5814 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1941-03-04)
No-MM_K5815 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1914-02-25)
No-MM_K5816 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1915-08-03)
No-MM_K5817 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1909-03-29)
No-MM_K5818 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1908-10-14)
No-MM_K5819 1 author (Christian Gierløff) addressed to: Edvard Munch
Date type: instance (1905-12-13)
No-MM_K5820 1 author (Christian Gierløff) address

CorrespDict

for item in CorrespDict:
    i=1
    print(item+",",CorrespDict[item]['date'])
    for author in CorrespDict[item]['authors']:
        print(i,author)
        i+=1
    print("\n")

# Main Registry

In [10]:
if os.path.isfile("register_tei.xml"):
    RegDict = collections.defaultdict(dict)
    print("RegDict initiated")
    print("Melting register_tei.xml")
    with open("register_tei.xml", "r", encoding="utf-8") as file: # Open a file
        tei = file.readlines() # Les innholdet som linjer
        tei = "".join(tei) # Linjene blir kombinert i en variabel
    soup = BeautifulSoup(tei, from_encoding="UTF-8", features="xml") # It is now soup
    # Don't worry about the error screaming about Unicode markup being provided twice
    print("Registry is now soup.")
    comments = 0
    commentDocs = 0
    for comment in soup.findAll(string=lambda text: isinstance(text, Comment)):
        if "xml:id=\"" in comment:
            commentDocs+=1
        comment.extract()
        comments+=1
    if comments > 0:
        print("Destroyed",comments,"<!--comments-->, of which",commentDocs,"contained an @XML:ID.")
    # ... and checking it twice.
    comments = soup.findAll(string=lambda text: isinstance(text, Comment))
    if comments:
        print("There are still",len(comments),"comments present.")
    else:
        print("All comments destroyed.")
        
    print("\nInitializing documentID scan.")
    documentIDs = []
    for document in soup.findAll("div", {"xml:id":True}):

        # Look for the document type assignment.
        documentType = document.find("list", {"type" : "objectType"}).findChild(True, recursive=True)#.attrs['n']
        # Checks if the words "letter" or "brev" appear in the type
        if "brev" in documentType or "letter" in documentType: 
            # Get the document ID from the <div> element.
            documentID = list(document.attrs.values())[0]
            documentIDs.append(documentID)
    print("Acquired",len(documentIDs),"documents classed as letters.\n")
        
    # Slight repetition - breaking DRY, I know - but by referencing documentIDs we're 100% only treating letters,
    # not wasting time on irrelevant documents
    i=1
    for eachID in documentIDs:
    
        
        i+=1
        #MM_K3421
        docAuthors = [] # List of authors to be included in the dict.
        docRecipRefs = [] # List of authors' reference URLs.
        docRecipients = [] # List of recipients to be included in the dict. 

        #print(eachID)
        # Munch is the author of everything in the registry.
        author = "Edvard Munch"

        # Target the document as var "document"
        document = soup.find("div", {"xml:id":eachID})

        # Target the recipients(s) as recipNameList
        recipNameList = document.find("item", {"n":"recipient"}).findChildren(True, recursive=True)
        X=0
        for name in recipNameList:
            try:
                recipName = recipNameList[X].contents[0]
                try:
                    targetRef = recipName['target']
                except:
                    targetRef = "N/A"
            except:
                recipName = "N/A"
                targetRef = "N/A"
            
            # Data cleaning
            recipName = recipName.replace(","," ")
            #recipName = recipName.replace(";"," ")
            #recipName = recipName.replace("["," ")
            #recipName = recipName.replace("]"," ")
            #recipName = recipName.replace("?"," ")
            recipName = re.sub(' +', ' ',recipName)
            recipName = recipName.strip()
            
            X+=1
            docRecipients.append(recipName)
            docRecipRefs.append(targetRef)

        statusMessage = str(eachID)+" "+str(len(docRecipients))+" recipients"
        if len(docRecipients) > 1:
            statusMessage += "s"
        statusMessage+=" ("
        z = 1
        for recipient in docRecipients:
            if recipient != "N/A":
                statusMessage+=str(recipient)
            else:
                statusMessage+="N/A (error?)"
            if z < len(docRecipients):
                statusMessage+=","
            z+=1
        statusMessage+=") from: "+str(author)


        isDocumentUndated = document.find("item", {"n":"undated"})
        if isDocumentUndated:
            # Document is straight up undated.
            date = "s.d."
            datetype = "N/A"

        else:
            #statusMessage+="\n>Dated: "
            isDocumentFromTo = document.find("date", {"from":True}) # Does the date element have a from assignment? 
            # ! Using "from" because PN1350 does not have a fromTo attr despite using fromTo. Uses "from", though. Works fine.
            if isDocumentFromTo: # If it does, and thus has a range (JK, No-MM_T1296 has FROM attr but not a TO attr.)
                doesDocumentHaveToDate = document.find("date", {"to":True})
                #statusMessage+="range, "
                if doesDocumentHaveToDate:
                    # Both from and to attributes are present.
                    fromDate = isDocumentFromTo['from'] # Extract 'from' date. 
                    toDate = isDocumentFromTo['to'] # Extract 'to' date.
                    datetype = "fromTo"
                    date = str(fromDate)+"%"+str(toDate)
                    #statusMessage+=datetype+" "+str(fromDate)+"-"+str(toDate)
                else:
                    # If the 'from' attribute is present without the 'to', it's interpreted as "not before this date".
                    date = isDocumentFromTo['from']
                    fromDate = isDocumentFromTo['from']
                    datetype = "notBefore"

            else: # If it doesn't:
                #statusMessage+="instance, "
                yearSent = document.find("date", {"type":"year","when":True}) # Check for year element
                monthSent = document.find("date", {"type":"month","when":True}) # Check for month element
                daySent = document.find("date", {"type":"day","when":True}) # Check for day element
                if yearSent:
                    datetype = "instance"
                    date = yearSent.attrs["when"]
                    if monthSent: # Only look for a month if there's a year. That 1 letter with just month/day, tho...
                        M = re.sub('[-]', '', monthSent.attrs["when"]) # Strip the random '-' characters in here.
                        date+="-"+str(M) # Join month to year by YYYY-MM.
                        if daySent: # Only applies if there is a month AND a day. No point having a day if you don't have a month.
                            M = re.sub('[-]', '', daySent.attrs["when"]) # Strip the random '-' characters in here, too.
                            date+="-"+str(M) # Join day to year-month by YYYY-MM-DD.

                else: 
                # If it doesn't have a year, make one last check
                    doesDocumentHaveToDate = document.find("date", {"to":True}) # if the date just has a to date...

                    if doesDocumentHaveToDate:
                    # If the 'to' attribute is present without the 'from', it's interpreted as "not after this date".
                        datetype = "notAfter"
                        date = doesDocumentHaveToDate['to']

                    else:
                    # All else has failed. This data is expunged.
                        datetype = "N/A"
                        date = "s.d."


        #statusMessage+="\n"
        if datetype == "N/A":
            statusMessage+="\n(undated)"
        else:
            statusMessage+="\nDate type: "+str(datetype)+" ("+str(date)+")"
        print(statusMessage)
        RegDict[eachID]['authors'] = author
        RegDict[eachID]['date'] = date
        RegDict[eachID]['datetype'] = datetype
        RegDict[eachID]['recipients'] = docRecipients
        
        json_object = json.dumps(RegDict, indent=4)

        with open("registry.json", "w") as outfile:
            outfile.write(json_object)

else:
    print("No correspondence.xml file provided. MXML will not munch letters to Munch.")

RegDict initiated
Melting register_tei.xml
Registry is now soup.
Destroyed 18 <!--comments-->, of which 3 contained an @XML:ID.
All comments destroyed.

Initializing documentID scan.
Acquired 2711 documents classed as letters.

No-MM_N0552 1 recipients (Kai Christensen) from: Edvard Munch
Date type: instance (1910-04-26)
No-MM_N0711 1 recipients (Uidentifisert person) from: Edvard Munch
Date type: instance (1933-12-31)
No-MM_N0712 1 recipients (Ludvig M Larssen) from: Edvard Munch
Date type: instance (1905)
No-MM_N0716 1 recipients (Christian Munch) from: Edvard Munch
Date type: instance (1874-06-18)
No-MM_N0717 1 recipients (Christian Munch) from: Edvard Munch
Date type: instance (1875-08-02)
No-MM_N0718 1 recipients (Christian Munch) from: Edvard Munch
Date type: instance (1878-05-18)
No-MM_N0719 1 recipients (Christian Munch) from: Edvard Munch
Date type: instance (1885-05-01)
No-MM_N0720 1 recipients (Christian Munch) from: Edvard Munch
Date type: instance (1889-02-01)
No-MM_N0721 

No-MM_N0803 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894-03-13)
No-MM_N0804 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894-04-29)
No-MM_N0805 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894-10-07)
No-MM_N0806 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894-10-24)
No-MM_N0807 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894-11-14)
No-MM_N0808 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1895-02-03)
No-MM_N0809 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1894)
No-MM_N0810 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1895-03)
No-MM_N0811 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1895)
No-MM_N0812 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1895-10)
No-MM_N0813 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instan

No-MM_N0894 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905-11-06)
No-MM_N0895 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905-11-17)
No-MM_N0896 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905-12-26)
No-MM_N0897 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905)
No-MM_N0898 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905)
No-MM_N0899 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905)
No-MM_N0900 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1906)
No-MM_N0901 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905-01-18)
No-MM_N0902 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1905)
No-MM_N0903 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1906-02-15)
No-MM_N0904 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1906-03-

No-MM_N0987 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1915-11-06)
No-MM_N0988 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1915-11-19)
No-MM_N0989 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1920-01-31)
No-MM_N0990 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1920-12-23)
No-MM_N0991 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1920)
No-MM_N0992 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1909-11)
No-MM_N0993 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1921-01-29)
No-MM_N0994 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1921-12-24)
No-MM_N0995 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1922-12-27)
No-MM_N0996 1 recipients (Karen Bjølstad) from: Edvard Munch
Date type: instance (1911-12-30-12-30)
No-MM_N0997 1 recipients (Karen Bjølstad) from: Edvard Munch
Da

No-MM_N1101 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1896-04-24)
No-MM_N1102 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1901-12-14)
No-MM_N1103 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1901)
No-MM_N1104 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1902-04-08)
No-MM_N1105 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1908-08-13)
No-MM_N1106 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1913-03-18)
No-MM_N1107 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1926-09-14)
No-MM_N1108 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1921-01-04)
No-MM_N1109 1 recipients (Inger Munch) from: Edvard Munch
Date type: fromTo (1916%1926)
No-MM_N1110 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1922-02-06)
No-MM_N1111 1 recipients (Inger Munch) from: Edvard Munch
Date type: instance (1923)
No-MM_N1112 1 

...

PN0868 1 recipients (Olav Paulsen) from: Edvard Munch
Date type: instance (1883)
PN0871 1 recipients (Olav Paulsen) from: Edvard Munch
Date type: instance (1919)
PN0872 1 recipients (Halfdan Roede) from: Edvard Munch
(undated)
PN0873 1 recipients (Halfdan Roede) from: Edvard Munch
(undated)
PN0874 1 recipients (Halfdan Roede) from: Edvard Munch
(undated)
PN0875 1 recipients (Halfdan Roede) from: Edvard Munch
(undated)
PN0876 1 recipients (Halfdan Roede) from: Edvard Munch
(undated)
PN0877 1 recipients (Halfdan Roede) from: Edvard Munch
Date type: instance (1922-04-22)
PN0878 1 recipients (Halfdan Roede) from: Edvard Munch
Date type: instance (1935-02-06)
PN0879 1 recipients (Halfdan Roede) from: Edvard Munch
Date type: instance (1943-12-30)
PN0880 1 recipients (Johannes Roede) from: Edvard Munch
(undated)
PN0881 1 recipients (Johannes Roede) from: Edvard Munch
Date type: instance (1932-06-03)
PN0883 1 recipients (Helga Rogstad) from: Edvard Munch
Date type: instance (1928-01-05)
PN0884

PN0979 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-08-16)
PN0980 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-10-12)
PN0981 1 recipients (Den Frie Udstilling) from: Edvard Munch
Date type: instance (1926-10-16)
PN0982 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-10-21)
PN0983 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-10-29)
PN0984 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-11-08)
PN0985 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-11-15)
PN0986 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
(undated)
PN0987 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-12-01)
PN0988 1 recipients (Städtische Kunsthalle ) from: Edvard Munch
Date type: instance (1926-12-08)
PN0989 1 recipients (Städtische Kunsthalle ) from: Edvar

PN1076 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-03-28)
PN1077 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-04-18)
PN1078 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-04-22)
PN1079 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-04-22)
PN1080 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-05-08)
PN1081 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-05-13)
PN1082 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-05-16)
PN1083 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-05-16)
PN1084 2 recipientss (Wilhelm Wartmann,Kunsthaus Zürich) from: Edvard Munch
Date type: instance (1922-05-18)
PN1085 2 recipients

PN1166 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-09-14)
PN1167 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-09-21)
PN1168 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-09-25)
PN1169 1 recipients (Ernest Thiel) from: Edvard Munch
(undated)
PN1170 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-10-11)
PN1171 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-10-22)
PN1172 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-10-26)
PN1173 1 recipients (Ernest Thiel) from: Edvard Munch
(undated)
PN1174 1 recipients (Ernest Thiel) from: Edvard Munch
(undated)
PN1175 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-11-26)
PN1176 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1907-12-02)
PN1177 1 recipients (Ernest Thiel) from: Edvard Munch
Date type: instance (1908-02-10)
PN1178 1 recipients (Erne

PN1272 1 recipients (Gustav Schiefler) from: Edvard Munch
Date type: instance (1928-12-02)
PN1273 1 recipients (Gustav Schiefler) from: Edvard Munch
Date type: instance (1909-01-25)
PN1274 1 recipients (Heinrich Eduard  Linde-Walther) from: Edvard Munch
(undated)
PN1276 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1903-03-06)
PN1277 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1906-07-19)
PN1278 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1905-06-18)
PN1279 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1920-10-31)
PN1280 1 recipients (Max Linde) from: Edvard Munch
(undated)
PN1282 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1932-02-05)
PN1283 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1931-12-20)
PN1284 1 recipients (Max Linde) from: Edvard Munch
Date type: instance (1935-01-06)
PN1285 1 recipients (Uidentifisert person) from: Edvard Munch
Date type: instance (1939